<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/main/Rentabilidad_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones previas

In [464]:
#!pip install xlsxwriter
#!pip install fastexcel

# Librerías necesarias

In [465]:
import polars as pl
import pandas as pd
import numpy as np
import xlsxwriter

In [466]:
archivo = 'Rentabilidad Amazon.xlsx'

# Archivos necesarios

In [467]:
renglonaje_og = pl.read_excel("Renglonaje.xlsx", sheet_id=3)
renglonaje = renglonaje_og.select(['código', 'clave', 'Total que paga Edy a Truper contando reserva en garantía CON IVA'])
renglonaje = renglonaje.rename({'Total que paga Edy a Truper contando reserva en garantía CON IVA':'Costo', 'código':'clave', 'clave':'sku'})

## Relación Asín-SKU

In [468]:
asin = pl.read_excel('Asin.xlsx', sheet_id=2)
asin = asin.select(['sku', 'asin', 'Sku Truper'])
asin2 = pl.read_excel('Asin.xlsx', sheet_id=3)
asin2 = asin2.select(['sku', 'asin', 'Sku Truper'])
asin = asin.vstack(asin2)
asin2 = pl.read_excel('Asin.xlsx', sheet_id=4)
asin2 = asin2.select(['sku', 'asin', 'Sku Truper'])
asin = asin.vstack(asin2)
asin2 = pl.read_excel('Asin.xlsx', sheet_id=5)
asin2 = asin2.select(['sku', 'asin', 'Sku Truper'])
asin = asin.vstack(asin2)
asin = asin.unique(subset='sku', keep = 'first')

In [469]:
asin = asin.rename({'sku':'alfa', 'Sku Truper':'sku'})
asin = asin.select(['sku', 'alfa', 'asin'])

In [470]:
renglonaje = renglonaje.join(asin, on='sku', how='left')

In [471]:
renglonaje = renglonaje.filter(pl.col('Costo').is_not_null())


In [472]:
c_sku = renglonaje.select(['sku', 'Costo'])
c_alfa = renglonaje.select(['alfa', 'Costo'])
c_alfa = c_alfa.rename({'alfa':'sku'})
c_asin = renglonaje.select(['asin', 'Costo'])
c_asin = c_asin.rename({'asin':'sku'})
c_clave = renglonaje.select(['clave', 'Costo'])
c_clave = c_clave.rename({'clave':'sku'})
c_clave = c_clave.with_columns(pl.col('sku').cast(pl.Utf8))

In [473]:
c_sku = c_sku.unique(subset='sku', keep = 'first')
c_alfa = c_alfa.unique(subset='sku', keep = 'first')
c_asin = c_asin.unique(subset='sku', keep = 'first')
c_clave = c_clave.unique(subset='sku', keep = 'first')

# Leyendo Reportes

Si solo se va a leer un reporte, este deberá ser guardado como "Reporte". Si no, los reportes deberán ser guardados de forma seriada, es decir, "Reporte1", "Reporte2", "Reporte3", etc.

El orden no debería importar

In [474]:
reportes = pl.read_csv('Reportes.txt', separator='\t')

In [475]:
duplicados = reportes.group_by('amazon-order-id').len()
display(duplicados.filter(pl.col('len') > 1))

amazon-order-id,len
str,u32
"""701-9053270-2310620""",2
"""702-3152486-5615458""",2


# Leyendo Transacciones

Si solo se va a leer una transacción, esta deberá ser guardada como "Transacciones". Si no, deberán ser guardadas de forma seriada, como los reportes.

***Nota 1: Las transacciones deberán ser pasados primero por la herramienta de limpieza de transacciones. Si no, generarán problemas***

***Nota 2: Los reembolsos se tratarán aparte***

In [476]:
transacciones = pl.read_excel('Transacciones.xlsx', sheet_id=1)

In [477]:
transacciones = transacciones.filter(pl.col('Tipo de transacción') != 'Reembolso')

In [478]:
transacciones = transacciones.rename({'Número de pedido':'amazon-order-id'})

In [479]:
df = reportes.join(transacciones, on='amazon-order-id', how='left')

Renombramos la columna de Número de pedido a amazon-amazon-amazon-amazon-amazon-order-id, para hacer la unión luego con el reporte de pedidos

In [480]:
formato_num = {pl.Int64:"0", pl.Float64:"0.00"}

# Leyendo los reportes de envíos

In [481]:
envios = pl.read_csv("Envios.csv", infer_schema_length=10000)

# Juntando el reporte de envíos y las transacciones

In [482]:
envios = envios.rename({'NUMERO ORDEN':'amazon-order-id'})
envios = envios.select(['amazon-order-id', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'PROTECCION ($)', 'TOTAL GUIA ($)'])

In [483]:
df = df.join(envios, on='amazon-order-id', how='left')

In [484]:
df = df.with_columns(pl.col('VALOR GUIA ($)').fill_null(0),
                       pl.col('ZONA EXTENDIDA ($)').fill_null(0),
                       pl.col('EXTRAGRANDE ($)').fill_null(0),
                       pl.col('SOBREPESOS ($)').fill_null(0),
                       pl.col('PROTECCION ($)').fill_null(0),
                       pl.col('TOTAL GUIA ($)').fill_null(0))

In [485]:
df = df.join(c_sku, on='sku', how='left')
df = df.update(c_alfa, on='sku', how='left', include_nulls=False)
df = df.update(c_asin, on='sku', how='left', include_nulls=False)
df = df.update(c_clave, on='sku', how='left', include_nulls=False)

In [486]:
c_asin = c_asin.rename({'sku':'asin'})
df = df.update(c_asin, on = 'asin', how='left', include_nulls=False)

In [487]:
df = df.with_columns(pl.col('shipping-price').fill_null(0),
                     pl.col('shipping-tax').fill_null(0),
                     pl.col('item-tax').fill_null(0),
                     pl.col('gift-wrap-price').fill_null(0),
                     pl.col('gift-wrap-tax').fill_null(0),
                     pl.col('item-promotion-discount').fill_null(0),
                     pl.col('ship-promotion-discount').fill_null(0))

In [488]:
df = df.with_columns((pl.col('TOTAL GUIA ($)') + pl.col('Costo de envío')).alias('TOTAL GUIA ($)'))
df = df.drop(['Costo de envío'])

In [489]:
df = df.with_columns((pl.col('item-price') - pl.col('item-tax') - pl.col('TOTAL GUIA ($)') - pl.col('Costo')).alias('Rentabilidad bruta'))

In [490]:
df = df.with_columns(pl.col('Rentabilidad bruta').round(2))

In [491]:
df = df.with_columns((pl.when(pl.col('order-status') == 'Cancelled'))
                        .then(None)
                        .otherwise(pl.col('Rentabilidad bruta'))
                        .alias('Rentabilidad bruta'))

In [492]:
df = df.with_columns((pl.when(pl.col('order-status') == 'Cancelled'))
                        .then(None)
                        .otherwise(pl.col('Costo'))
                        .alias('Costo'))

In [493]:
df = df.with_columns((pl.when(pl.col('Rentabilidad bruta') > 0))
                     .then(pl.lit('Sí'))
                     .otherwise(pl.lit('No'))
                     .alias('¿Rentable?'))

In [494]:
df = df.with_columns(pl.col('Rentabilidad bruta').round(2),
                     pl.col('TOTAL GUIA ($)').round(2),
                     pl.col('Costo').round(2))

In [495]:
df = df.with_columns((pl.when(pl.col('order-status') == 'Cancelled'))
                        .then(pl.lit('Cancelado'))
                        .otherwise(pl.col('¿Rentable?'))
                        .alias('¿Rentable?'))

# Reembolsos
Los reembolsos se verán en otra hoja

In [496]:
t_reembolsos = pl.read_excel('Transacciones.xlsx', sheet_id=1)
t_reembolsos = t_reembolsos.filter(pl.col('Tipo de transacción') == 'Reembolso')
t_reembolsos = t_reembolsos.rename({'Número de pedido':'amazon-order-id'})

# Escribiendo Excel

In [497]:
wb = xlsxwriter.Workbook(archivo)

In [498]:
df.write_excel(workbook=wb, worksheet='Rentabilidad')
t_reembolsos.write_excel(workbook=wb, worksheet='Reembolsos')

In [499]:
wb.close()